In [1]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.metrics import mean_absolute_error, mean_squared_error
from scipy.signal import medfilt
from sklearn import decomposition
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import MinMaxScaler

from tqdm import tqdm

from itertools import product

import pickle

import matplotlib.pyplot as plt
%matplotlib inline

c:\Users\gioel\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\gioel\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
c:\Users\gioel\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, Dense, BatchNormalization, Activation, Dropout, TimeDistributed
def arch(data,val):
    input_dots = Input((40,19))

    x = Dense(200)(input_dots)
    x = BatchNormalization()(x)
    x = Activation('elu')(x)

    bottleneck = Dense(32, activation='linear')(x)


    x = Dense(200)(bottleneck)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    out = Dense(19, activation='linear')(x)

    model = Model(input_dots, out)
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001), loss="mse")
    model.summary()

    callback = keras.callbacks.EarlyStopping(monitor='val_loss', patience=30)

    history = model.fit(
        data,
        data,
        validation_data=(val,val),
        epochs=200,
        batch_size=32,
        verbose=1,
        callbacks=[callback]
    )
    return history, model

In [3]:
WINDOW_SIZE=40
def create_sequences(values, time_steps=WINDOW_SIZE):
    output = []
    for i in range(len(values) - time_steps + 1):
        output.append(values[i : (i + time_steps)])
    return np.stack(output)

In [8]:
with open(f'./DATA_SPLITTED/app_tot.pkl', 'rb') as f:
    DATA = pickle.load(f)

train=DATA['X_train']
val=DATA['X_val']


history,model=arch(train,val)

model.save_weights(f"MODEL_Dense/model_app_Win_{WINDOW_SIZE}.h5")


Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 40, 19)]          0         
                                                                 
 dense_8 (Dense)             (None, 40, 200)           4000      
                                                                 
 batch_normalization_4 (Batc  (None, 40, 200)          800       
 hNormalization)                                                 
                                                                 
 activation_4 (Activation)   (None, 40, 200)           0         
                                                                 
 dense_9 (Dense)             (None, 40, 32)            6432      
                                                                 
 dense_10 (Dense)            (None, 40, 200)           6600      
                                                           

In [4]:
def MODEL():
    input_dots = Input((40,19))

    x = Dense(200)(input_dots)
    x = BatchNormalization()(x)
    x = Activation('elu')(x)

    bottleneck = Dense(32, activation='linear')(x)


    x = Dense(200)(bottleneck)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    out = Dense(19, activation='linear')(x)

    model = Model(input_dots, out)
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001), loss="mse")
    return model

In [5]:
with open(f'./DATA_SPLITTED/app_tot.pkl', 'rb') as f:
            DATA = pickle.load(f)
    
model=MODEL()

model.load_weights(filepath=f"MODEL_Dense/model_app_Win_{WINDOW_SIZE}.h5")

e=model.predict(DATA['X_test'])
score=np.mean(np.power(e-DATA['X_test'],2),axis=(1,2))

np.save('./score_analisi/DENSE_AE.npy',score,allow_pickle=True)

In [11]:
import csv


header=['th_factor','method','value','F1','precision','recall','TP','TN','FP','FN']

with open('Dense_AE.csv', 'w', encoding='UTF8') as f:
    writer = csv.writer(f)

    writer.writerow(header)
    thresholding_factor=[0.5,1,1.5,2]
    for t in thresholding_factor:
        with open(f'./DATA_SPLITTED/app_tot.pkl', 'rb') as f:
            DATA = pickle.load(f)
    
        model=MODEL()

        model.load_weights(filepath=f"MODEL_Dense/model_app_Win_{WINDOW_SIZE}.h5")

        e=model.predict(DATA['X_test'])
        score=np.mean(np.power(e-DATA['X_test'],2),axis=(1,2))

        #IQR
        q1, q3 = np.percentile(score, 25), np.percentile(score, 75)
        iqr = q3 - q1
        IQR = q3 + t* iqr

        #MAD
        median = np.median(score)
        mad = 1.4826 * np.median(np.abs(score - median))
        MAD = median + t * mad

        #STD
        mean, std = np.mean(score), np.std(score)
        STD = mean + t * std

        method=[IQR,MAD,STD]
        
        for g in range(len(method)):
            TP=0
            TN=0
            FN=0
            FP=0

            for i in tqdm(range(1,11)):
                if(i!=7):
                
                            TEST=np.load(f'./OUTPUTS_ROOT/data/processed/spark_0_15s/spark_0_trace-scl_std/test{i}.npy',allow_pickle=True)

                            ANOMALY=np.load(f'./OUTPUTS_ROOT/data/processed/spark_0_15s/spark_0_trace-scl_std/y_test{i}.npy',allow_pickle=True)

                            with open(f'./OUTPUTS_ROOT/data/interim/spark_0_15s/test_info{i}.pkl', 'rb') as f:
                                TEST_info= pickle.load(f)

                            
                            for x in range(len(TEST_info)):

                                X=create_sequences(TEST[x])

                                Y=model.predict(X)


                                error=np.mean(np.power(X-Y,2),axis=(1,2))
                                error=[error[l] if error[l]<2 else 2 for l in range(len(error))]

                                outlier=error>method[g]

                                le=len(outlier)
                                
                                true_= ANOMALY[x][20:le+20]>=1
                                prediction_ = outlier[:].astype(int)==1

                                TP = TP+(true_ & prediction_).sum()   
                                TN = TN+(~true_ & ~prediction_).sum()  
                                FP = FP+(~true_ & prediction_).sum()    
                                FN = FN+(true_ & ~prediction_).sum()    

            PREC=TP / (TP + FP)
            REC = TP/ (TP+FN)
            f1=2 * PREC * REC/(PREC + REC)
            m=['IQR','MAD','STD']
            row=[t,m[g],method[g],f1,PREC,REC,TP,TN,FP,FN]
            writer.writerow(row)

100%|██████████| 10/10 [00:07<00:00,  1.25it/s]
